# **3060. User Activities within Time Bounds**

## **Schema**

### **Table: Sessions**
```sql
+---------------+----------+
| Column Name   | Type     |
+---------------+----------+
| user_id       | int      |
| session_start | datetime |
| session_end   | datetime |
| session_id    | int      |
| session_type  | enum     |
+---------------+----------+
```
- `session_id` is a column of unique values for this table.
- `session_type` is an ENUM (category) type of (`Viewer`, `Streamer`).
- This table contains user ID, session start, session end, session ID, and session type.

## **Problem Statement**

Write a solution to find the users who have had at least one consecutive session of the same type (either `Viewer` or `Streamer`) with a maximum gap of 12 hours between sessions.

Return the result table ordered by `user_id` in ascending order.


## **Example**

### **Input:**

#### **Sessions Table:**
```sql
+---------+---------------------+---------------------+------------+--------------+
| user_id | session_start       | session_end         | session_id | session_type | 
+---------+---------------------+---------------------+------------+--------------+
| 101     | 2023-11-01 08:00:00 | 2023-11-01 09:00:00 | 1          | Viewer       |  
| 101     | 2023-11-01 10:00:00 | 2023-11-01 11:00:00 | 2          | Streamer     |   
| 102     | 2023-11-01 13:00:00 | 2023-11-01 14:00:00 | 3          | Viewer       | 
| 102     | 2023-11-01 15:00:00 | 2023-11-01 16:00:00 | 4          | Viewer       | 
| 101     | 2023-11-02 09:00:00 | 2023-11-02 10:00:00 | 5          | Viewer       | 
| 102     | 2023-11-02 12:00:00 | 2023-11-02 13:00:00 | 6          | Streamer     | 
| 101     | 2023-11-02 13:00:00 | 2023-11-02 14:00:00 | 7          | Streamer     | 
| 102     | 2023-11-02 16:00:00 | 2023-11-02 17:00:00 | 8          | Viewer       | 
| 103     | 2023-11-01 08:00:00 | 2023-11-01 09:00:00 | 9          | Viewer       | 
| 103     | 2023-11-02 20:00:00 | 2023-11-02 23:00:00 | 10         | Viewer       | 
| 103     | 2023-11-03 09:00:00 | 2023-11-03 10:00:00 | 11         | Viewer       | 
+---------+---------------------+---------------------+------------+--------------+
```

### **Output:**
```sql
+---------+
| user_id |
+---------+
| 102     |
| 103     |
+---------+
```

### **Explanation:**
- **User ID 101**: Will not be included in the final output as they do not have any consecutive sessions of the same session type.
- **User ID 102**: Will be included as they had two `Viewer` sessions with session IDs 3 and 4, respectively, and the time gap between them was less than 12 hours.
- **User ID 103**: Participated in two `Viewer` sessions with a gap of less than 12 hours between them, identified by session IDs 10 and 11. Therefore, user 103 will be included in the final output.

### **Output Table:**
```sql
+---------+
| user_id |
+---------+
| 102     |
| 103     |
+---------+
```


**Solution-1:**
``` sql
select distinct s1.user_id
from Sessions s1
join Sessions s2
on s1.user_id = s2.user_id and s1.session_type = s2.session_type and
timestampdiff(hour,s1.session_end, s2.session_start) <= 12 and S1.session_start < S2.session_start
order by user_id asc

**Solution-2:**
``` sql
-- get next session (of same type) info with window function
WITH NEXT_SESSION_INFO AS (
    SELECT *,
           LEAD(session_start) OVER(PARTITION BY user_id, session_type ORDER BY session_start ASC) AS next_session_start
    FROM Sessions
)
SELECT DISTINCT user_id
FROM NEXT_SESSION_INFO
WHERE TIMESTAMPDIFF(HOUR, session_end, next_session_start) <= 12
ORDER BY user_id ASC